In [1]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

import warnings
import functools
import wrds
# Ignore all warnings
warnings.filterwarnings("ignore")

In [109]:
df_prices = pd.read_parquet('stock_price.parquet')
df_prices

,date,ticker,prc,bid,ask,permno,permco,comnam
0,2017-12-01,JJSF,150.07001,150.06000,150.07001,10026,7976,J & J SNACK FOODS CORP
1,2017-12-01,RMCF,11.77000,11.71000,11.77000,10044,7992,ROCKY MOUNTAIN CHOC FAC INC NEW
2,2017-12-04,RMCF,11.87000,11.72000,11.89000,10044,7992,ROCKY MOUNTAIN CHOC FAC INC NEW
3,2017-12-05,RMCF,11.91000,11.71000,11.91000,10044,7992,ROCKY MOUNTAIN CHOC FAC INC NEW
4,2017-12-06,RMCF,11.84000,11.83000,11.89000,10044,7992,ROCKY MOUNTAIN CHOC FAC INC NEW
...,...,...,...,...,...,...,...,...
7713931,2022-08-22,TSLA,869.73999,869.97998,870.15997,93436,53453,TESLA INC
7713932,2022-08-23,TSLA,889.35999,889.28998,889.38000,93436,53453,TESLA INC
7713933,2022-08-24,TSLA,891.28998,891.28998,891.50000,93436,53453,TESLA INC
7713934,2022-08-25,TSLA,296.07001,296.00000,296.13000,93436,53453,TESLA INC


In [110]:
df_ED = pd.read_parquet('tickers_ED_20182023.parquet')
df_ED

,Short_ticker,Name,Ticker,Date,Event Type
0,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER
1,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER
2,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER
3,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER
4,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER
...,...,...,...,...,...
19471,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER
19472,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER
19473,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER
19474,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER


In [111]:
# manually filter duplicate ticker's row
# only 5 tickers have duplicate rows: AR, LEN, MKC, STZ, WSO

# 1. drop Ticker = "AR CN" in df_ED
df_ED = df_ED[df_ED['Ticker'] != 'AR CN']

# 2. LEN: permno != 89731
df_prices = df_prices[df_prices['permno'] != 89731]

# 3. MKC: permno != 89155
df_prices = df_prices[df_prices['permno'] != 89155]

# 4. STZ: permno != 64899
df_prices = df_prices[df_prices['permno'] != 64899]

#5. WSO: permno != 46068
df_prices = df_prices[df_prices['permno'] != 46068]

In [112]:
# filter stock prices to only include those in the ED list
unique_tickers = df_ED['Short_ticker'].unique()
df_prices = df_prices[df_prices['ticker'].isin(unique_tickers)]
df_prices.sort_values(by=['ticker', 'date'], inplace=True)
df_prices.reset_index(drop=True, inplace=True)
df_prices

,date,ticker,prc,bid,ask,permno,permco,comnam
0,2017-12-01,A,68.69000,68.67,68.68000,87432,36364,AGILENT TECHNOLOGIES INC
1,2017-12-04,A,66.24000,66.23,66.24000,87432,36364,AGILENT TECHNOLOGIES INC
2,2017-12-05,A,65.83000,65.80,65.81000,87432,36364,AGILENT TECHNOLOGIES INC
3,2017-12-06,A,65.89000,65.89,65.90000,87432,36364,AGILENT TECHNOLOGIES INC
4,2017-12-07,A,66.71000,66.71,66.72000,87432,36364,AGILENT TECHNOLOGIES INC
...,...,...,...,...,...,...,...,...
1138846,2023-12-22,ZTS,194.98000,194.95,194.98000,13788,54327,ZOETIS INC
1138847,2023-12-26,ZTS,195.50000,195.53,195.53999,13788,54327,ZOETIS INC
1138848,2023-12-27,ZTS,196.89999,196.98,197.00000,13788,54327,ZOETIS INC
1138849,2023-12-28,ZTS,197.16000,197.27,197.28000,13788,54327,ZOETIS INC


In [113]:
# include stock prices for 5 days before or after the ED date

df_ED['Date'] = pd.to_datetime(df_ED['Date'])
df_ED['Date'] = df_ED['Date'].dt.date
df_ED['Date'] = df_ED['Date'].astype(str)
df_prices['date'] = df_prices['date'].astype(str)
# df_prices['date'] = pd.to_datetime(df_prices['date'])


In [114]:

result = pd.DataFrame(columns=['date', 'permno','prc', 'bid', 'ask', 'ticker', 'comnam'])

for ticker in df_ED['Short_ticker'].unique():
    date_list = df_ED[df_ED['Short_ticker'] == ticker]['Date'].tolist()

    ticker_prices = df_prices[df_prices['ticker'] == ticker]
    matched_rows = ticker_prices[ticker_prices['date'].isin(date_list)]

    # prices on ED date have to be greater than 20
    matched_rows = matched_rows[matched_rows['prc'] >= 20]

    indices = matched_rows.index.tolist()
    
    for index in indices:
        start_index = max(0, index - 5)
        end_index = min(index + 5, ticker_prices.index.max())
        selected_rows = df_prices.iloc[start_index:end_index+1]

        # Concatenate the selected_rows dataframe with the result_df dataframe
        result = pd.concat([result, selected_rows])

result.reset_index(inplace=True,drop=True)

In [115]:
result

,date,permno,prc,bid,ask,ticker,comnam,permco
0,2018-02-07,87432,68.06,68.06000,68.07000,A,AGILENT TECHNOLOGIES INC,36364.0
1,2018-02-08,87432,65.05,65.03000,65.05000,A,AGILENT TECHNOLOGIES INC,36364.0
2,2018-02-09,87432,66.98,66.97000,66.98000,A,AGILENT TECHNOLOGIES INC,36364.0
3,2018-02-12,87432,68.43,68.40000,68.41000,A,AGILENT TECHNOLOGIES INC,36364.0
4,2018-02-13,87432,68.34,68.33000,68.34000,A,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...,...
147109,2023-11-03,13788,162.23,162.19000,162.23000,ZTS,ZOETIS INC,54327.0
147110,2023-11-06,13788,163.17,163.20000,163.27000,ZTS,ZOETIS INC,54327.0
147111,2023-11-07,13788,167.16,167.24001,167.28000,ZTS,ZOETIS INC,54327.0
147112,2023-11-08,13788,170.87,170.89000,170.92999,ZTS,ZOETIS INC,54327.0


In [116]:
result['date'] = pd.to_datetime(result['date'])
result[result['ticker'] == 'AAPL']

,date,permno,prc,bid,ask,ticker,comnam,permco
836,2018-01-25,14593,171.11000,171.10001,171.11000,AAPL,APPLE INC,7.0
837,2018-01-26,14593,171.50999,171.39000,171.39999,AAPL,APPLE INC,7.0
838,2018-01-29,14593,167.96001,167.95000,167.96001,AAPL,APPLE INC,7.0
839,2018-01-30,14593,166.97000,166.96001,166.97000,AAPL,APPLE INC,7.0
840,2018-01-31,14593,167.42999,167.45000,167.47000,AAPL,APPLE INC,7.0
...,...,...,...,...,...,...,...,...
1084,2023-11-03,14593,176.64999,176.63000,176.64000,AAPL,APPLE INC,7.0
1085,2023-11-06,14593,179.23000,179.28999,179.30000,AAPL,APPLE INC,7.0
1086,2023-11-07,14593,181.82001,181.87000,181.89999,AAPL,APPLE INC,7.0
1087,2023-11-08,14593,182.89000,182.89999,182.91000,AAPL,APPLE INC,7.0


In [117]:
result.to_parquet('stock_price_ED_20182023.parquet')